In [1]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta


In [6]:
import warnings
warnings.filterwarnings("ignore")


In [7]:
# ticker_list = [
#     "RELIANCE.NS", "HDFCBANK.NS", "ICICIBANK.NS", "INFY.NS", "TCS.NS",
#     "LT.NS", "ITC.NS", "KOTAKBANK.NS", "HINDUNILVR.NS", "AXISBANK.NS",
#     "SBIN.NS", "BHARTIARTL.NS", "BAJFINANCE.NS", "ASIANPAINT.NS", "MARUTI.NS",
#     "HCLTECH.NS", "SUNPHARMA.NS", "NTPC.NS", "TITAN.NS", "ONGC.NS",
#     "ULTRACEMCO.NS", "POWERGRID.NS", "ADANIENT.NS", "JSWSTEEL.NS", "WIPRO.NS",
#     "TATAMOTORS.NS", "INDUSINDBK.NS", "BAJAJFINSV.NS", "HINDALCO.NS", "GRASIM.NS",
#     "TECHM.NS", "DIVISLAB.NS", "NESTLEIND.NS", "TATASTEEL.NS", "CIPLA.NS",
#     "ADANIPORTS.NS", "DRREDDY.NS", "BPCL.NS", "SBILIFE.NS", "BRITANNIA.NS",
#     "EICHERMOT.NS", "HEROMOTOCO.NS", "COALINDIA.NS", "BAJAJ-AUTO.NS", "HDFCLIFE.NS",
#     "SHREECEM.NS", "APOLLOHOSP.NS", "M&M.NS", "ICICIPRULI.NS", "UPL.NS", "ABB.NS", "ACC.NS", "ADANIGREEN.NS", "ALKEM.NS",
#     "AMBUJACEM.NS", "AUROPHARMA.NS", "BAJAJHLDNG.NS", "BANDHANBNK.NS", "BANKBARODA.NS",
#     "BERGEPAINT.NS", "BIOCON.NS", "BOSCHLTD.NS", "CANBK.NS", "CHOLAFIN.NS",
#     "COLPAL.NS", "DABUR.NS", "DIXON.NS", "GAIL.NS", "GODREJCP.NS",
#     "HAVELLS.NS", "ICICIGI.NS", "ICICIPRULI.NS", "IDEA.NS", "IGL.NS",
#     "INDIGO.NS", "INDUSTOWER.NS", "IOC.NS", "LICHSGFIN.NS", "MARICO.NS", "MOTHERSON.NS", "MRF.NS", "NHPC.NS", "NMDC.NS",
#     "PAGEIND.NS", "PEL.NS", "PIDILITIND.NS", "PNB.NS", "RECLTD.NS",
#     "SAIL.NS", "SIEMENS.NS", "SRF.NS", "TORNTPHARM.NS", "TRENT.NS",
#     "TVSMOTOR.NS", "UBL.NS", "VEDL.NS", "VOLTAS.NS", "ZEEL.NS"
# ]

# Input_date='2025-07-15'


In [34]:
ticker_list = [
    "5PAISA.NS",
    "AAVAS.NS",
    "APTUS.NS",
    "ARMANFIN.NS",
    "ASIANPAINT.NS",
    "BANDHANBNK.NS",
    "DELTACORP.NS",
    "HINDUNILVR.NS",
    "INFY.NS",
    "JIOFIN.NS",   # Assumed ticker for Jio Financial Services — please verify
    "LALPATHLAB.NS",
    "TATAELXSI.NS",
    "TCS.NS",
    "WIPRO.NS",
    "SWIGGY.NS",
      "^NSEI",         # Nifty 50
    "^NSEBANK",      # Nifty Bank
    "^NSEMDCP50",    # Nifty Midcap 50
    "^CNX100",       # Nifty 100
    "^CNXIT",        # Nifty IT
    "^CNXFMCG",      # Nifty FMCG
    "^CNXAUTO",      # Nifty Auto
    "^CNXPHARMA",    # Nifty Pharma
    "^CNXREALTY",    # Nifty Realty
    "^CNXENERGY"

]
Input_date='2025-07-15'

In [35]:
def process_stock(ticker):
    Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
    
    Data.columns = Data.columns.droplevel(0)
    # Data.head(5)
    Data=Data[0:-20]
    ## Add Useful KPIs
    
    Data['RSI']=ta.rsi(Data["Close"], length=14)
    
    info = yf.Ticker(ticker).info
    Data['Sector']=info.get("sector")
    Data['Industry']=info.get("industry")
    
    MACD=ta.macd(Data["Close"], fast=12, slow=26, signal=9)
    Data=pd.concat([Data,MACD],axis=1)
    
    Data.rename(columns={
        "MACD_12_26_9": "MACD_Line",
        "MACDs_12_26_9": "MACD_Signal",
        "MACDh_12_26_9": "MACD_Histogram"
    }, inplace=True)
    
    
    Data["EMA_20"] = ta.ema(Data["Close"], length=20)
    Data["EMA_50"] = ta.ema(Data["Close"], length=50)
    
    # Add Bollinger Bands (20-day default)
    bbands = ta.bbands(Data["Close"], length=20)
    
    # Combine with main DataFrame
    Data = pd.concat([Data, bbands], axis=1)
    
    # Optional rename (if needed)
    Data.rename(columns={
        "BBL_20_2.0": "BB_Lower",
        "BBM_20_2.0": "BB_Middle",
        "BBU_20_2.0": "BB_Upper",
        "BBB_20_2.0": "BB_Bandwidth",
        "BBP_20_2.0": "BB_Percent"
    }, inplace=True)
    
    # Add ADX (default 14-period)
    adx_df = ta.adx(Data["High"], Data["Low"], Data["Close"], length=14)
    
    # Join it with main DataFrame
    Data = pd.concat([Data, adx_df], axis=1)
    
    # Optional rename (for clarity)
    Data.rename(columns={
        "ADX_14": "ADX",
        "DMP_14": "DI_Plus",
        "DMN_14": "DI_Minus"
    }, inplace=True)
    
    # Add ATR (default length=14)
    Data["ATR_14"] = ta.atr(Data["High"], Data["Low"], Data["Close"], length=14)
    
    # Add Stochastic Oscillator (default k=14, d=3)
    stoch_df = ta.stoch(Data["High"], Data["Low"], Data["Close"], k=14, d=3)
    
    # Combine with main DataFrame
    Data = pd.concat([Data, stoch_df], axis=1)
    
    # Optional rename
    Data.rename(columns={
        "STOCHk_14_3_3": "Stoch_%K",
        "STOCHd_14_3_3": "Stoch_%D"
    }, inplace=True)
    
    
    
    # Add CCI (default period = 20)
    Data["CCI_20"] = ta.cci(Data["High"], Data["Low"], Data["Close"], length=20)


    
    Data['DayOfWeek'] = Data.index.dayofweek
    Data['Month'] = Data.index.month
    # Data.to_csv("test.csv")
    
    Data.reset_index(inplace=True)
    
    ## Feature Engineering
    
    Data['Volume1']=Data['Volume'].shift(1)
    Data['Volume_Inc']=(Data['Volume']-Data['Volume1'])/Data['Volume1']
    
    Data['EMA_20_Price']=(Data['Close']-Data['EMA_20'])/Data['EMA_20']
    Data['EMA_50_Price']=(Data['Close']-Data['EMA_50'])/Data['EMA_50']
    
    Data['Stoch_k_D']=(Data['Stoch_%K']-Data['Stoch_%D'])
    
    Data['Max_Close_Next_10'] = Data['Close'].shift(-1).rolling(window=10, min_periods=1).max()
    
    # Calculate the percentage increase from the current close to the maximum future close
    Data['Close_Inc_Max'] = (Data['Max_Close_Next_10'] - Data['Close']) / Data['Close']
    
    # Define the target variable
    Data['Buy'] = Data['Close_Inc_Max'].apply(lambda x: 1 if x >= 0.05 else 0)
    
    ## Data Clean
    
    Clean_Data=Data[['RSI', 'MACD_Histogram','BB_Percent','ADX','ATR_14', 'Stoch_%K',
           'Stoch_%D', 'CCI_20','Volume_Inc', 'EMA_20_Price',
           'EMA_50_Price', 'Stoch_k_D', 'Buy','DayOfWeek','Month']]
    
    Clean_Data=Clean_Data.dropna(subset=['RSI', 'MACD_Histogram','BB_Percent','ADX','ATR_14', 'Stoch_%K',
           'Stoch_%D', 'CCI_20','Volume_Inc', 'EMA_20_Price',
           'EMA_50_Price','DayOfWeek','Month'])
    
    # Clean_Data.head(10)
    
    ## Visualize
    
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    features = ["RSI", "MACD_Histogram", "BB_Percent", "ADX", "ATR_14", 
                "Stoch_%K", "CCI_20", "EMA_20_Price", "Volume_Inc",'DayOfWeek','Month']
    
    # for feature in features:
    #     plt.figure(figsize=(6, 3))
    #     sns.kdeplot(data=Clean_Data[Clean_Data["Buy"] == 1], x=feature, label="Buy = 1", shade=True)
    #     sns.kdeplot(data=Clean_Data[Clean_Data["Buy"] == 0], x=feature, label="Buy = 0", shade=True)
    #     plt.title(f"{feature} vs Buy")
    #     plt.legend()
    #     plt.tight_layout()
    #     plt.show()
    
    
    # for feature in features:
    #     plt.figure(figsize=(6, 3))
    #     sns.boxplot(data=Clean_Data, x="Buy", y=feature)
    #     plt.title(f"{feature} by Buy value")
    #     plt.tight_layout()
    #     plt.show()
    
    
    Clean_Data.drop(columns="Volume_Inc",inplace=True)
    
    # # Step 1: Compute correlation matrix
    # corr_matrix = Clean_Data.corr()
    
    # # Step 2: Set up the heatmap
    # plt.figure(figsize=(12, 8))
    # sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
    # plt.title("Correlation Heatmap")
    # plt.tight_layout()
    # plt.show()
    
    # Clean_Data.head(5)
    
    ## Model Build
    
    from sklearn.preprocessing import StandardScaler
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(Clean_Data.drop("Buy", axis=1))
    
    
    
    # X_scaled
    
    from sklearn.model_selection import train_test_split
    y=Clean_Data['Buy']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # ### Logisitic
    
    # from sklearn.linear_model import LogisticRegression
    
    # model=LogisticRegression()
    # model.fit(X_train, y_train)
    # y=model.predict(X_test)
    # y_proba = model.predict_proba(X_test)[:, 1]
    # from sklearn.metrics import classification_report,roc_auc_score
    # print("🔹 Logistic Regression:")
    # print(classification_report(y_test, y))
    # print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")
    
    # ### Random Forest
    
    # from sklearn.ensemble import RandomForestClassifier
    
    # model=RandomForestClassifier(random_state=42)
    # model.fit(X_train, y_train)
    # y=model.predict(X_test)
    # y_proba = model.predict_proba(X_test)[:, 1]
    # feature_importances = model.feature_importances_
    # features = Clean_Data.drop("Buy", axis=1).columns
    # # Combine into a DataFrame
    # F_Imp = pd.DataFrame({
    #     "Feature": features,
    #     "Importance": feature_importances
    # }).sort_values("Importance", ascending=False)
    # print(F_Imp)
    # from sklearn.metrics import classification_report
    # print("🔹 RandomForestClassifier:")
    # print(classification_report(y_test, y))
    # print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")
    
    ### XG Boost
    
    from xgboost import XGBClassifier
    model=XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
    model.fit(X_train, y_train)
    y=model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    feature_importances = model.feature_importances_
    features = Clean_Data.drop("Buy", axis=1).columns
    # Combine into a DataFrame
    F_Imp = pd.DataFrame({
        "Feature": features,
        "Importance": feature_importances
    }).sort_values("Importance", ascending=False)
    # print(F_Imp)
    from sklearn.metrics import classification_report
    # print("🔹 XGBClassifier: ")
    # print(classification_report(y_test, y))
    # print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")
    
    
    
    data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
    data_Test.columns = data_Test.columns.droplevel(0)
    
    data_Test['RSI']=ta.rsi(data_Test["Close"], length=14)
    
    info = yf.Ticker(ticker).info
    data_Test['Sector']=info.get("sector")
    data_Test['Industry']=info.get("industry")
    
    MACD=ta.macd(data_Test["Close"], fast=12, slow=26, signal=9)
    data_Test=pd.concat([data_Test,MACD],axis=1)
    
    data_Test.rename(columns={
        "MACD_12_26_9": "MACD_Line",
        "MACDs_12_26_9": "MACD_Signal",
        "MACDh_12_26_9": "MACD_Histogram"
    }, inplace=True)
    
    
    data_Test["EMA_20"] = ta.ema(data_Test["Close"], length=20)
    data_Test["EMA_50"] = ta.ema(data_Test["Close"], length=50)
    
    # Add Bollinger Bands (20-day default)
    bbands = ta.bbands(data_Test["Close"], length=20)
    
    # Combine with main DataFrame
    data_Test = pd.concat([data_Test, bbands], axis=1)
    
    # Optional rename (if needed)
    data_Test.rename(columns={
        "BBL_20_2.0": "BB_Lower",
        "BBM_20_2.0": "BB_Middle",
        "BBU_20_2.0": "BB_Upper",
        "BBB_20_2.0": "BB_Bandwidth",
        "BBP_20_2.0": "BB_Percent"
    }, inplace=True)
    
    # Add ADX (default 14-period)
    adx_df = ta.adx(data_Test["High"], data_Test["Low"], data_Test["Close"], length=14)
    
    # Join it with main DataFrame
    data_Test = pd.concat([data_Test, adx_df], axis=1)
    
    # Optional rename (for clarity)
    data_Test.rename(columns={
        "ADX_14": "ADX",
        "DMP_14": "DI_Plus",
        "DMN_14": "DI_Minus"
    }, inplace=True)
    
    # Add ATR (default length=14)
    data_Test["ATR_14"] = ta.atr(data_Test["High"], data_Test["Low"], data_Test["Close"], length=14)
    
    # Add Stochastic Oscillator (default k=14, d=3)
    stoch_df = ta.stoch(data_Test["High"], data_Test["Low"], data_Test["Close"], k=14, d=3)
    
    # Combine with main DataFrame
    data_Test = pd.concat([data_Test, stoch_df], axis=1)
    
    # Optional rename
    data_Test.rename(columns={
        "STOCHk_14_3_3": "Stoch_%K",
        "STOCHd_14_3_3": "Stoch_%D"
    }, inplace=True)
    
    
    
    # Add CCI (default period = 20)
    data_Test["CCI_20"] = ta.cci(data_Test["High"], data_Test["Low"], data_Test["Close"], length=20)

    data_Test['DayOfWeek'] = data_Test.index.dayofweek
    data_Test['Month'] = data_Test.index.month
    
    data_Test['Volume1']=data_Test['Volume'].shift(1)
    data_Test['Volume_Inc']=(data_Test['Volume']-data_Test['Volume1'])/data_Test['Volume1']
    
    data_Test['EMA_20_Price']=(data_Test['Close']-data_Test['EMA_20'])/data_Test['EMA_20']
    data_Test['EMA_50_Price']=(data_Test['Close']-data_Test['EMA_50'])/data_Test['EMA_50']
    
    data_Test['Stoch_k_D']=(data_Test['Stoch_%K']-data_Test['Stoch_%D'])
    
    data_Test['Close1']=data_Test['Close'].shift(10)
    data_Test['Close_Inc']=(data_Test['Close1']-data_Test['Close'])/data_Test['Close']
    
    data_Test.reset_index(inplace=True)
    data_Test=data_Test[data_Test['Date']==Input_date]
    
    Clean_Data_Test=data_Test[['RSI', 'MACD_Histogram','BB_Percent','ADX','ATR_14', 'Stoch_%K',
           'Stoch_%D', 'CCI_20','Volume_Inc', 'EMA_20_Price',
           'EMA_50_Price', 'Stoch_k_D','DayOfWeek','Month']]
    
    # Step 1: Get feature columns (same as used during training)
    features = ['RSI', 'MACD_Histogram', 'BB_Percent', 'ADX', 'ATR_14',
                'Stoch_%K', 'Stoch_%D', 'CCI_20', 'EMA_20_Price', 'EMA_50_Price', 'Stoch_k_D','DayOfWeek','Month']
    
    # Step 2: Extract today's row (last row in the test table)
    today_row = Clean_Data_Test[features].iloc[-1:]  # Keep it as DataFrame
    
    # Step 3: Apply the same scaler used during training
    today_scaled = scaler.transform(today_row)
    
    # Step 4: Predict using the trained model
    buy_prediction = model.predict(today_scaled)[0]
    print(ticker)
    buy_probability = model.predict_proba(today_scaled)[0][1]
    
    # Step 5: Print result
    # print("🔮 Buy Prediction:", "Yes" if buy_prediction == 1 else "No")
    # print(f"📊 Confidence: {buy_probability:.2%}")
    return {
            "Ticker": ticker,
            "Buy": "Yes" if buy_prediction == 1 else "No",
            "Confidence": round(buy_probability * 100, 2)
        }
    
    


In [36]:
results = []

for ticker in ticker_list:
    result = process_stock(ticker)
    results.append(result)

final_df = pd.DataFrame(results)


print(final_df)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


5PAISA.NS


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


AAVAS.NS


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

^CNXPHARMA


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


^CNXREALTY


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


^CNXENERGY
           Ticker  Buy  Confidence
0       5PAISA.NS   No        4.84
1        AAVAS.NS   No        4.37
2        APTUS.NS   No        0.44
3     ARMANFIN.NS  Yes       97.36
4   ASIANPAINT.NS   No        0.01
5   BANDHANBNK.NS   No        1.46
6    DELTACORP.NS   No        0.12
7   HINDUNILVR.NS   No        0.01
8         INFY.NS   No       25.22
9       JIOFIN.NS   No        0.31
10  LALPATHLAB.NS   No        0.03
11   TATAELXSI.NS   No        0.04
12         TCS.NS  Yes       97.78
13       WIPRO.NS   No       15.76
14      SWIGGY.NS   No       35.05
15          ^NSEI   No        0.47
16       ^NSEBANK   No        0.02
17     ^NSEMDCP50   No        0.05
18        ^CNX100   No        0.04
19         ^CNXIT  Yes       83.46
20       ^CNXFMCG   No        0.01
21       ^CNXAUTO   No        0.01
22     ^CNXPHARMA   No        0.01
23     ^CNXREALTY   No        2.30
24     ^CNXENERGY   No        0.23


In [37]:
from datetime import date


filename = f"{Input_date}_Screener_India.csv"

final_df.to_csv(filename, index=False)